In [1]:
import pandas as pd
import numpy as np
import json
import torch

# Direct imports from the same directory
from tabddpm_adapter import TabDDPMAdapter
from tabddpm_benchmark import evaluate_tabddpm, print_evaluation_results
from tabddpm_utils import preprocess_data, get_tstr_results

In [2]:
# Load configuration
print("# Load configuration")
with open("config.json", "r") as f:
    config = json.load(f)

# Load configuration


In [3]:
# 1. Load and prepare the dataset
print("# 1. Load and prepare the dataset")
data_raw = pd.read_csv("nursery.csv")
print(f"Columns in dataset: {data_raw.columns.tolist()}")
print(f"Dataset shape: {data_raw.shape}")
print(data_raw.head())

# 1. Load and prepare the dataset
Columns in dataset: [' parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'Target']
Dataset shape: (12960, 9)
   parents has_nurs      form children     housing     finance         social  \
0    usual   proper  complete        1  convenient  convenient        nonprob   
1    usual   proper  complete        1  convenient  convenient        nonprob   
2    usual   proper  complete        1  convenient  convenient        nonprob   
3    usual   proper  complete        1  convenient  convenient  slightly_prob   
4    usual   proper  complete        1  convenient  convenient  slightly_prob   

        health     Target  
0  recommended  recommend  
1     priority   priority  
2    not_recom  not_recom  
3  recommended  recommend  
4     priority   priority  


In [4]:
# 2. Preprocess data and detect categorical columns
print("\n# 2. Preprocess data and detect categorical columns")
data, categorical_columns = preprocess_data(data_raw)
print(f"Detected categorical columns: {categorical_columns}")

2025-04-27 11:53:26,930 - INFO - Converted  parents to category type (has 3 unique values)
2025-04-27 11:53:26,932 - INFO - Converted has_nurs to category type (has 5 unique values)
2025-04-27 11:53:26,935 - INFO - Converted form to category type (has 4 unique values)
2025-04-27 11:53:26,935 - INFO - Converted children to category type (has 4 unique values)
2025-04-27 11:53:26,935 - INFO - Converted housing to category type (has 3 unique values)
2025-04-27 11:53:26,940 - INFO - Converted finance to category type (has 2 unique values)
2025-04-27 11:53:26,943 - INFO - Converted social to category type (has 3 unique values)
2025-04-27 11:53:26,946 - INFO - Converted health to category type (has 3 unique values)
2025-04-27 11:53:26,948 - INFO - Converted Target to category type (has 5 unique values)



# 2. Preprocess data and detect categorical columns
Detected categorical columns: [' parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'Target']


In [5]:
# 3. Define the target column for this dataset
print("\n# 3. Define the target column for this dataset")
target_column = "Target"
print(f"Target column: {target_column}")


# 3. Define the target column for this dataset
Target column: Target


In [6]:
# 4. Split the data into features and target
print("\n# 4. Split the data into features and target")
X = data.drop(columns=[target_column])
y = data[target_column]
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target distribution:\n{y.value_counts()}")


# 4. Split the data into features and target
Features shape: (12960, 8)
Target shape: (12960,)
Target distribution:
Target
not_recom     4320
priority      4266
spec_prior    4044
very_recom     328
recommend        2
Name: count, dtype: int64


In [7]:
# 5. Initialize and train TabDDPM
print("\n# 5. Initialize and train TabDDPM")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tabddpm = TabDDPMAdapter(**config["tabddpm_params"], device=device)
print("Training TabDDPM model...")
tabddpm.fit(X, y)
print("Training completed")


# 5. Initialize and train TabDDPM
Training TabDDPM model...
Original data shape: (12960, 9), Target column: Target
Added OneHotEncoder for 8 categorical columns
Target 'Target' identified as categorical with 5 classes
Class mapping: {'not_recom': 0, 'priority': 1, 'recommend': 2, 'spec_prior': 3, 'very_recom': 4}

Preprocessing Summary:
- Number of numerical features: 0
- Number of categorical features: 8
- Categorical columns: [' parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']
- Target column: Target
- Target type: Categorical
- Number of target classes: 5
X shape: (12960, 8), y shape: (12960,)
X_tensor shape: torch.Size([12960, 27]), y_tensor shape: torch.Size([12960])
X_tensor shape: torch.Size([12960, 27])
y_tensor shape: torch.Size([12960])


Epoch 299, Loss: 0.3398: 100%|██████████| 300/300 [10:22<00:00,  2.08s/it]

Training completed


In [8]:
# 6. Generate synthetic data
print("\n# 6. Generate synthetic data")
n_samples = 1000  
print(f"Generating {n_samples} synthetic samples...")
synthetic_data = tabddpm.generate(n_samples)
print(f"Generated {len(synthetic_data)} synthetic samples")
print("Synthetic data head:")
print(synthetic_data.head())


# 6. Generate synthetic data
Generating 1000 synthetic samples...
Sample timestep    0
Sample timestep    0
Sample timestep    0
Sample timestep    0
Debug - out_dict type: <class 'torch.Tensor'>
Added target column 'Target' with 5 unique values
Final columns in synthetic data: [' parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health', 'Target']
Generated 1000 synthetic samples
Synthetic data head:
       parents     has_nurs        form children     housing     finance  \
0  pretentious     improper  incomplete     more   less_conv  convenient   
1   great_pret     improper      foster        2    critical      inconv   
2  pretentious    very_crit      foster        2   less_conv  convenient   
3        usual    very_crit  incomplete        1  convenient  convenient   
4   great_pret  less_proper  incomplete        2    critical      inconv   

          social       health      Target  
0  slightly_prob    not_recom  very_recom  
1  slightly_prob     pri

In [9]:
# 7. Evaluate quality using TSTR and other metrics
print("\n# 7. Evaluate quality using TSTR and other metrics")
print("Running evaluation...")
evaluation_results = evaluate_tabddpm(data, synthetic_data, target_column=target_column)
print_evaluation_results(evaluation_results)


2025-04-27 12:06:27,938 - WARNING - No numerical columns found for likelihood fitness evaluation
2025-04-27 12:06:27,954 - INFO - Encoded categorical target with mapping: {'not_recom': 0, 'priority': 1, 'recommend': 2, 'spec_prior': 3, 'very_recom': 4}



# 7. Evaluate quality using TSTR and other metrics
Running evaluation...


2025-04-27 12:06:36,354 - ERROR - Error training XGBoost classifier: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3], got [0 1 3 4]
2025-04-27 12:06:36,375 - INFO - Encoded categorical targets with mapping: {'not_recom': 0, 'priority': 1, 'recommend': 2, 'spec_prior': 3, 'very_recom': 4}
c:\Users\kamal\OneDrive\Documents\Vilo\DEAKIN\TRIMESTER 1 2025\SIT378 TEAM PROJECT B\KatabaticGitRepo\sit378\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\kamal\OneDrive\Documents\Vilo\DEAKIN\TRIMESTER 1 2025\SIT378 TEAM PROJECT B\KatabaticGitRepo\sit378\lib\site-packages\xgboost\core.py:158: UserWarning: [12:06:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

 

In [10]:
# 8. Extract and display TSTR results specifically
print("\n# 8. TSTR Performance Results")
tstr_results = get_tstr_results(evaluation_results)
if tstr_results is not None:
    print(tstr_results)


# 8. TSTR Performance Results
                    Accuracy        F1
LogisticRegression  0.152855  0.186176
RandomForest        0.176157  0.214634
MLP                 0.174074  0.214376
XGBoost             0.179475  0.217605


In [11]:
# 9. Save the synthetic data
print("\n# 9. Save synthetic data")
output_path = "nursery_synthetic.csv"
synthetic_data.to_csv(output_path, index=False)
print(f"Synthetic data saved to {output_path}")

print("\nTest completed successfully!")


# 9. Save synthetic data
Synthetic data saved to nursery_synthetic.csv

Test completed successfully!
